In [1]:
from polymesh.space import StandardFrame, PointCloud
from neumann.linalg.vector import Vector
from neumann.linalg import linspace
from polymesh.utils import center_of_points
import numpy as np

In [2]:
Lx, Ly, Lz = 300, 300, 300
points_per_edge = 3
mesh_size = Lx / (points_per_edge-1)

In [3]:
points = []
nTotalPoints = 0  # node counter

# corners
corner_coords = [
    [-Lx/2, -Ly/2, -Lz/2],
    [Lx/2, -Ly/2, -Lz/2],
    [Lx/2, Ly/2, -Lz/2],
    [-Lx/2, Ly/2, -Lz/2],
    [-Lx/2, -Ly/2, Lz/2],
    [Lx/2, -Ly/2, Lz/2],
    [Lx/2, Ly/2, Lz/2],
    [-Lx/2, Ly/2, Lz/2]
]
corner_coords = np.array(corner_coords)
points.append(corner_coords)
nTotalPoints += len(corner_coords)

# populate edges
nodes_of_edges = [
    [0, 1], [1, 2], [2, 3], [3, 0],
    [4, 5], [5, 6], [6, 7], [7, 4],
    [0, 4], [1, 5], [2, 6], [3, 7]
    ]
edge_coords = []
N = points_per_edge + 2
for nodes in nodes_of_edges:
    p0 = corner_coords[nodes[0]]
    p1 = corner_coords[nodes[1]]
    edge_coords.append(linspace(p0, p1, N)[1:-1])
edge_coords = np.vstack(edge_coords)
points.append(edge_coords)
nTotalPoints += len(edge_coords)

# faces
corners_of_faces = {
    'front' : [1, 2, 6, 5], 
    'back' : [0, 3, 7, 4], 
    'left' : [2, 3, 7, 6],  
    'right' : [0, 1, 5, 4],
    'bottom' : [0, 1, 2, 3], 
    'top' : [4, 5, 6, 7],  
}
edges_of_faces = {
    'front' : [1, 5, 9, 10], 
    'back' : [3, 7, 8, 11], 
    'right' : [0, 9, 4, 8],  
    'left' : [2, 6, 10, 11],
    'bottom' : [0, 1, 2, 3], 
    'top' : [4, 5, 6, 7],  
}

# center of face
def cof(id) : return center_of_points(corner_coords[corners_of_faces[id]])

# face frames
frames = {}
frames['front'] = StandardFrame(dim=3, origo=cof('front'))
rot90z = 'Body', [0, 0, np.pi/2], 'XYZ'
frames['left'] = frames['front'].fork(*rot90z).move(cof('left') - cof('front'))
frames['back'] = frames['left'].fork(*rot90z).move(cof('back') - cof('left'))
frames['right'] = frames['back'].fork(*rot90z).move(cof('right') - cof('back'))
rot_front_top = 'Body', [0, -np.pi/2, 0], 'XYZ'
frames['top'] = frames['front'].fork(*rot_front_top).move(cof('top') - cof('front'))
rot180y = 'Body', [0, np.pi, 0], 'XYZ'
frames['bottom'] = frames['top'].fork(*rot180y).move(cof('bottom') - cof('top'))

In [4]:
frames['left'].origo()

array([  0., 150.,   0.])

In [5]:
coords = np.array([[0, -Lx/2, 0], [0, Lx/2, -Ly/2], [0, Lx/2, 0], [0, Lx/2, Ly/2]])
coords

array([[   0., -150.,    0.],
       [   0.,  150., -150.],
       [   0.,  150.,    0.],
       [   0.,  150.,  150.]])

In [6]:
pc = PointCloud(coords, frame=frames['left'])
pc

VectorBase([[   0., -150.,    0.],
            [   0.,  150., -150.],
            [   0.,  150.,    0.],
            [   0.,  150.,  150.]])

In [7]:
pc.show()

VectorBase([[ 150.,  150.,    0.],
            [-150.,  150., -150.],
            [-150.,  150.,    0.],
            [-150.,  150.,  150.]])

In [8]:
pc.center()

array([-75., 150.,   0.])

## **Single Vector Test**

In [9]:
A = StandardFrame(dim=3)

In [10]:
v1 = Vector([Lx/2, Ly/2, 0.0], frame=A)
v1

VectorBase([150., 150.,   0.])

In [11]:
v1.show()

array([150., 150.,   0.])

In [12]:
arr2 = v1.show(frames['left'])
arr2

array([ 150., -150.,    0.])

In [13]:
v2 = Vector(arr2, frame=frames['left'])
v2

VectorBase([ 150., -150.,    0.])

In [14]:
v2.show()

array([150., 150.,   0.])

In [15]:
coords = np.array([v2.array]) - np.array([150., 0., 0.])
coords

array([[   0., -150.,    0.]])

In [16]:
v = Vector(coords[0], frame=frames['left'])
v.show()

array([ 1.50000000e+02, -9.18485099e-15,  0.00000000e+00])

In [17]:
pc = PointCloud(coords, frame=frames['left'])
pc.show()

VectorBase([[150., 150.,   0.]])

In [18]:
v = Vector(coords[0], frame=frames['left'])
v.show()

array([ 1.50000000e+02, -9.18485099e-15,  0.00000000e+00])

In [19]:
pc = PointCloud(coords, frame=frames['left'])
pc.show()

VectorBase([[150., 150.,   0.]])

In [20]:
frames['left'].origo()

array([  0., 150.,   0.])

In [21]:
frames['left'].origo(frames['left'])

array([0., 0., 0.])